In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import TruncatedSVD, NMF
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import pickle


def tokenize_stem(series):

    tokenizer =TreebankWordTokenizer()
    stemmer = PorterStemmer()
    series = series.apply(lambda x: x.replace("\n", ' '))
    series = series.apply(lambda x: tokenizer.tokenize(x))
    series = series.apply(lambda x: [stemmer.stem(w) for w in x])
    series = series.apply(lambda x: ' '.join(x))
    return series

def display_topics(model, feature_names, no_top_words, topic_names=None):
    '''
    displays topics and returns list of toppics
    '''

    topic_list = []
    for i, topic in enumerate(model.components_):
        if not topic_names or not topic_names[i]:
            print("\nTopic ", i)
        else:
            print("\nTopic: '",topic_names[i],"'")

        print(", ".join([feature_names[k]
                       for k in topic.argsort()[:-no_top_words - 1:-1]]))
        topic_list.append(", ".join([feature_names[k]
                       for k in topic.argsort()[:-no_top_words - 1:-1]]))
    return model.components_, topic_list

def return_topics(series, num_topics, no_top_words, model, vectorizer):
    '''
    returns document_topic matrix and topic modeling model
    '''
    #turn job into series
    series = tokenize_stem(series)
    #transform series into corpus
    ex_label = [e[:30]+"..." for e in series]
    #set vectorizer ngrams = (2,2)
    vec = vectorizer(stop_words = 'english')

    doc_word = vec.fit_transform(series)

    #build model
    def_model = model(num_topics)
    def_model = def_model.fit(doc_word)
    doc_topic = def_model.transform(doc_word)
    #print('model components: ', def_model.components_[0].shape)
    #print('doc_topic', doc_topic[0])
    model_components, topic_list = display_topics(def_model, vec.get_feature_names(), no_top_words)
    return def_model.components_, doc_topic, def_model, vec, topic_list#, topics


def process_data():
    '''
    uses the functions above to read in files, model, and return a topic_document dataframe
    '''
    #read in jobs file and get descriptions
    df = pd.read_csv('D:/Job_Recommender/ML_Project.csv')
    #df = df[df.keyword!='marketing']
    jobs_df = pd.DataFrame(zip(df['Job Description'], df['keyword']), columns = ['Description', 'Job'])

    array, doc, topic_model, vec, topic_list  = return_topics(jobs_df['Description'],20, 10, TruncatedSVD, TfidfVectorizer)

    topic_df = pd.DataFrame(doc)
    topic_df.columns = ['Topic ' + str(i+1) for i in range(len(topic_df.columns)) ]

    topic_df['job'] = jobs_df.Job
    #topic_df.to_csv('D:/Job_Recommender/topic_df2.csv')
    resume="Data enthusisast\n who is keen to work and do progress with capabilities of machine learning"
    resume=list(resume)
    resume = pd.Series(resume)
    doc = tokenize_stem(resume)
    doc = vec.transform(doc)
    #topic_model_name = 'D:/Job_Recommender/topic_model1.sav'
    #pickle.dump(topic_model, open(topic_model_name, 'wb'))
    vec_name = 'D:/Job_Recommender/job_vec1.sav'
    pickle.dump(vec, open(vec_name, 'wb'))
    return topic_df, topic_model, vec, topic_list

In [2]:
process_data()


Topic  0
charlott, deloitt, consult, scienc, data, ventur, red, lowe, senior, connect

Topic  1
data, tool, analyt, nc, scientist, year, experi, variou, serv, creat

Topic  2
variou, experi, year, valid, opportun, identifi, clean, optimizations, set, sourc

Topic  3
scientist, need, futur, lead, becom, skill, ll, narr, mining, discov

Topic  4
data, busi, perform, narr, told, explor, present, discov, mining, nc

Topic  5
work, year, experi, nc, relev, machin, learn, consult, team, requir

Topic  6
candid, technolog, appli, knowledg, manag, projects, autom, cognit, help, nc

Topic  7
collabor, product, cross, funct, manipul, access, team, applic, data, scienc

Topic  8
consult, deloitt, collabor, industri, product, funct, manipul, cross, access, work

Topic  9
consult, scienc, deloitt, data, ventur, red, senior, industri, lowe, engin

Topic  10
design, engin, nc, architectur, integr, process, compil, develop, requir, shadow

Topic  11
engin, work, avail, enrich, model, train, relev, te

C:\Users\avuda\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(     Topic 1   Topic 2   Topic 3   Topic 4   Topic 5   Topic 6   Topic 7  \
 0   0.021822  0.389852  0.826392  0.091642 -0.331294 -0.192427 -0.028135   
 1   0.026414  0.464419 -0.380196  0.661692 -0.354254  0.138416  0.029508   
 2   0.984186 -0.077162 -0.003351  0.003044 -0.028155 -0.030386  0.005283   
 3   0.013808  0.239761  0.270393 -0.077069  0.050714  0.427555 -0.141673   
 4   0.005876  0.103392  0.046249 -0.053524  0.068137  0.115412  0.074789   
 ..       ...       ...       ...       ...       ...       ...       ...   
 61  0.021822  0.389852  0.826392  0.091642 -0.331294 -0.192427 -0.028135   
 62  0.023338  0.419692 -0.356365  0.639826 -0.345047  0.142590  0.031244   
 63  0.038391  0.657445 -0.371126 -0.534549 -0.310064 -0.167776  0.007040   
 64  0.505384 -0.031343  0.009195 -0.008137  0.005115  0.138552  0.022094   
 65  0.452382 -0.013792  0.014948 -0.014776  0.015216  0.172789  0.022855   
 
      Topic 8   Topic 9  Topic 10  ...  Topic 12  Topic 13  Topic 14  Topi

In [20]:
def predictive_modeling(df):
    '''
    fits, optimizes, and predicts job class based on topic modeling corpus
    '''
    X,y = df.iloc[:,0:-1], df.iloc[:, -1]
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, shuffle=False)

    param_grid = {'n_estimators': [100,200,300,400,500], 'max_depth': [3,7,9, 11]}
    #search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
    #search.fit(X_tr, y_tr)
    #bp = search.best_params_
    #print(bp)
    #rfc = RandomForestClassifier(n_estimators = bp['n_estimators'], max_depth = bp['max_depth'])
    rfc = RandomForestClassifier(n_estimators =3500, max_depth = 10)
    rfc.fit(X_tr, y_tr)
    print('acc: ', np.mean(cross_val_score(rfc, X_tr, y_tr, scoring = 'accuracy', cv=5)))
    print('test_acc: ', accuracy_score(y_te, rfc.predict(X_te)))
    print(rfc.predict(X_te))
    return rfc

In [21]:
df=pd.read_csv("D:/Job_Recommender/topic_df2.csv")

print(df['job'].value_counts(ascending=True))
predictive_modeling(df)



acturial,data,scientist\t    22
data,analytics               26
data,scientist               27
senior,data,scientist\t      27
ai,ml,developer\t            30
Name: job, dtype: int64
acc:  0.7333333333333334
test_acc:  0.9259259259259259
['ai,ml,developer\t' 'ai,ml,developer\t' 'ai,ml,developer\t'
 'ai,ml,developer\t' 'ai,ml,developer\t' 'ai,ml,developer\t'
 'ai,ml,developer\t' 'ai,ml,developer\t' 'ai,ml,developer\t'
 'ai,ml,developer\t' 'ai,ml,developer\t' 'ai,ml,developer\t'
 'ai,ml,developer\t' 'ai,ml,developer\t' 'ai,ml,developer\t'
 'data,analytics' 'data,analytics' 'data,analytics' 'data,analytics'
 'data,analytics' 'data,analytics' 'data,analytics' 'data,analytics'
 'data,analytics' 'data,analytics' 'data,analytics' 'data,analytics']


RandomForestClassifier(max_depth=10, n_estimators=3500)